In [0]:
dbutils.widgets.text("booking_date", "2024-07-06")
booking_date = dbutils.widgets.get("booking_date") 

In [0]:
from pyspark.sql.functions import *
booking_data=f"/Volumes/workspace/zoom_car/zoom_car/zoom_car_bookings_{booking_date}.json"

In [0]:
booking_df = spark.read.option("multiline","true").json(booking_data)


In [0]:

def isComplete(df,colname):
        df_clean = df.filter(col(colname).isNotNull())
        return df_clean

In [0]:
booking_df = isComplete(booking_df, "booking_id")
booking_df = isComplete(booking_df, "customer_id")
booking_df = isComplete(booking_df, "car_id")
booking_df = isComplete(booking_df, "booking_date")

In [0]:
booking_df = booking_df.withColumn("valid_date", to_date("booking_date", "yyyy-MM-dd"))
booking_df = booking_df.filter(col("valid_date").isNotNull())
booking_df = booking_df.drop("valid_date")

In [0]:
valid_statuses = ["complete", "cancelled", "in_progress"]
booking_df = booking_df.filter(col("status").isin(valid_statuses))

In [0]:
booking_df = booking_df.withColumn("start_ts", to_timestamp("start_time"))
booking_df = booking_df.withColumn("end_ts", to_timestamp("end_time"))
booking_df = booking_df.withColumn("start_date", date_format(col("start_ts"), "yyyy-MM-dd"))
booking_df = booking_df.withColumn("start_time_only", date_format(col("start_ts"), "HH:mm:ss"))
booking_df = booking_df.withColumn("end_date", date_format(col("end_ts"), "yyyy-MM-dd"))
booking_df = booking_df.withColumn("end_time_only", date_format(col("end_ts"), "HH:mm:ss"))

In [0]:
booking_df = booking_df.withColumn("duration_minutes", round((col("end_ts").cast("long") - col("start_ts").cast("long")) / 60, 2))

In [0]:
# booking_df.write \
#     .format("delta") \
#     .mode("overwrite") \
#     .saveAsTable("default.zoom_car_bookings1")

In [0]:
from delta.tables import DeltaTable

In [0]:
delta_table = DeltaTable.forName(spark, "default.zoom_car_bookings1")
cancelled_df = booking_df.filter(col("status") == "cancelled")
active_df = booking_df.filter(col("status") != "cancelled")
delta_table.alias("target").merge(
    cancelled_df.alias("source"),
    "target.booking_id = source.booking_id AND target.customer_id = source.customer_id"
).whenMatchedDelete().execute()
delta_table.alias("target").merge(
    active_df.alias("source"),
    "target.booking_id = source.booking_id AND target.customer_id = source.customer_id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

In [0]:
df = spark.table("default.zoom_car_bookings1")
df.display()